# 类

## Public、private 与 protected 修饰符

#### 默认 public

In [1]:
class Animal {
    public name: string
    public constructor(name: string) {
        this.name = name
    }
    public move(distance: number) {
        console.log(`${this.name} moved ${distance}`)
    }
}

// 等于

class AnimalDefaultPublic {
    name: string
    constructor(name: string) {
        this.name = name
    }
    move(distance: number) {
        console.log(`${this.name} moved ${distance}`)
    }
}

undefined

#### private 修饰的属性，只能在声明它的类内部使用

In [2]:
class MyNameIsPrivate {
    private name: string
    constructor(name: string) {
        this.name = name
    }
    getName() {
        return this.name
    }
}

let mp = new MyNameIsPrivate('Tom')

undefined

In [3]:
console.log(mp.getName())

Tom


undefined

In [4]:
console.log(mp.name)

Error: Line 1, Character 16
console.log(mp.name)
_______________^
TS2341: Property 'name' is private and only accessible within class 'MyNameIsPrivate'.

TypeScript 以结构来比较不同类型。只要两个类型具有同样的结构，就认为两者是兼容的。

In [5]:
class Point {
    x: number
    y: number
    constructor(x: number, y: number) {
        this.x = x
        this.y = y
    }
}

class Coordinate {
    x: number
    y: number
    constructor(x: number, y: number) {
        this.x = x
        this.y = y
    }
}

let point = new Point(10, 10)
let coordinate = new Coordinate(20, 20)

point = coordinate

console.log(point)

Coordinate { x: 20, y: 20 }


undefined

但是如果两个类型中有一个有 `private` 或者 `protected` 修饰的类型，另一个必须有**来自一个地方声明的 `private` 或者 `protected` 类型**。

In [6]:
class Shape {
    private color: string
}

// OK
class Square extends Shape { }

undefined

In [7]:
// 不 OK，因为 Circle 的 private color 与 Shape 的不是同一个声明
class Circle extends Shape {
    private color: string
}

Error: Line 2, Character 7
class Circle extends Shape {
______^
TS2415: Class 'Circle' incorrectly extends base class 'Shape'.
  Types have separate declarations of a private property 'color'.

In [8]:
// 和 Shape 声明一样，但是因为有 private，两者不兼容
class CustomShape {
    private color: string
}

let shape = new Shape()
let customShape = new CustomShape()

shape = customShape

Error: Line 9, Character 1
shape = customShape
^
TS2322: Type 'CustomShape' is not assignable to type 'Shape'.
  Types have separate declarations of a private property 'color'.

#### protected 修饰的属性，和 private 类似，但是还可以在其派生类内使用

In [9]:
class Person {
    protected name: string
    private sex: number
    constructor(name: string) {
        this.name = name
    }
}

class Employee extends Person {
    private department: string
    constructor(name: string, department: string) {
        super(name);
        this.department = department;
    }
    
    introSelf() {
        // 可以访问 this.name，但是不可以访问 this.sex，因为它是 Person 的私有属性
        console.log(`Halo, my name is ${this.name}, I work in ${this.department}.`)
    }
}

let kobe = new Employee('Kobe Bryant', 'Lakers')
kobe.introSelf()

Halo, my name is Kobe Bryant, I work in Lakers.


undefined

constructor 构造函数也可以被标记为 protected，这意味着该类不能在其类声明块外进行实例化，但是可以扩展。

In [10]:
class AI {
    protected constructor() {}
    static getInstance() {
        return new AI()
    }
}

class Droid extends AI {
    constructor() {
        super()
    }
}

let r2d2 = new Droid()
console.log(r2d2)

Droid {}


undefined

In [11]:
// this is OK
let ai: AI = AI.getInstance()
console.log(ai)

AI {}


undefined

In [12]:
// this is not OK
let anotherAi = new AI()

Error: Line 2, Character 17
let anotherAi = new AI()
________________^
TS2674: Constructor of class 'AI' is protected and only accessible within the class declaration.

#### readonly 修饰符

可以用 readonly 来修饰属性，reaadonly 属性只能在声明时或者构造函数内初始化。

In [13]:
class Hero {
    readonly superPower: string
    readonly fromEarth: boolean = true
    constructor(superPower: string) {
        this.superPower = superPower
    }
}

undefined

#### 参数属性

上例中，我们的 superPower 写了两次，声明时和构造函数参数内。使用参数属性前缀，我们可以将声明和赋值结合到一个地方。该属性对 `private`、`public` 等同样有效。

In [14]:
class Book {
    constructor(public name: string, private pages: number, readonly author: string) {}
    
    getPages() {
        return this.pages
    }
    
    introSelf() {
        console.log(this)
    }
}

let book = new Book('Lord of the Rings', 666, 'J.R.R Tolkin')
book.introSelf()
console.log(book.getPages())

Book { name: 'Lord of the Rings', pages: 666, author: 'J.R.R Tolkin' }
666


undefined

## 存取器

setter 与 getter

注意上述例子中，为了获取 private 属性 pages，我们需要调用一个方法。使用 setter 和 getter 可以向属性一样来进行操作。

如果一个属性只有 get 而没有 set，会被自动推断为 readonly。

In [15]:
class Novel {
    private __progress: number
    constructor(public name: string, private __pages: number, readonly author: string) {
        this.__progress = 0
    }
    
    get pages(): number {
        return this.__pages
    }
    
    set progress(progress: number) {
        if (progress < 0) {
            console.log('Error, progress cant below 0.')
            return
        }
        this.__progress = progress
        console.log(`OK, current progress is ${progress}.`)
    }
}

let novel = new Novel('Lord of the Rings', 666, 'J.R.R Tolkin')

console.log(novel.pages)

novel.progress = 10
novel.progress = -2

666
OK, current progress is 10.
Error, progress cant below 0.


-2

In [16]:
// 只有 get，被自动推断为 readonly
novel.pages = 100

Error: Line 2, Character 7
novel.pages = 100
______^
TS2540: Cannot assign to 'pages' because it is a read-only property.

## 静态属性

静态属性为类本身而非实例的属性。

In [17]:
class Car {
    static maxSpeed: number = 100
    
    constructor(speed: number) {
        if (speed > Car.maxSpeed) {
            console.log('Error, speed too high')
        }
    }
}

let car = new Car(200)

Error, speed too high


100

## 抽象类

抽象类通常不能被实例化，而是作为其他类的基类。与接口不同的是，抽象类可以包含实现。抽象方法将不会被自动继承，必须在派生类内手动实现。

In [18]:
abstract class Container {
    storage: number

    abstract fill(): void

    empty(): void {
        this.storage = 0
    }
}

class Cup extends Container {
    storage = 0

    fill() {
        this.storage = 100
    }
    
    take(amount: number) {
        this.storage -= amount
    }
}

let cup = new Cup()
console.log(cup.storage)
cup.fill()
console.log(cup.storage)
cup.empty()
console.log(cup.storage)

0
100
0


undefined

## 关于类

当我们声明一个类的时候，我们产生了两个东西：

- 类的实例的类型（表示类的实例的类型）
- 构造函数

这两者都是类本身，参见下边<a href="#类中声明的-constructor-与-实例的-contructor-属性：">解释</a>

In [19]:
class Device {
    constructor() {}
}

// Device 可以作为一个类型使用
let device: Device

// constructor 会在 new 的时候被调用
device = new Device()

console.log(device)

Device {}


undefined

既然类可以生成类型，我们就可以把它**当做接口来使用。**

In [20]:
class Computer {
    brand: string
}

interface MacbookPro extends Computer {
    price: string
}

let myMacbookPro: MacbookPro = {
    brand: 'Apple',
    price: '$1000'
}

console.log(myMacbookPro)

{ brand: 'Apple', price: '$1000' }


undefined

#### 类中声明的 constructor 与 实例的 contructor 属性：

- 类中声明的 constructor 函数会被赋值给声明的类本身。
- 实例的 contructor 属性指向实例的原型，也就是类的 prototype 的 contructor 属性，也就是类本身
- 类的 constructor 属性指向 Function 类，因为类是 Function 的实例

##### 类中声明的 constructor 函数会被赋值给声明的类本身。

In [21]:
class Album {
    constructor(public artist: string) {
        this.artist = artist
        console.log('instance')
    }
    
    play() {
        console.log('Playing~~~')
    }
}

undefined

我们查看 TypeScript 生成的 JavaScript 代码，可以看到 Album 被赋值给了构造函数 Album:

```javascript
let Album = (function () {
    function Album(artist) {
        this.artist = artist
        console.log('instance')
    }
    Album.prototype.play = function() {
        console.log('Playing~~~')
    }
    return Album;
})()
```

##### 实例的 contructor 属性指向实例的原型，也就是类的 prototype 的 contructor 属性，也就是类本身

##### 类的 constructor 属性指向 Function 类，因为类是 Function 的实例

In [22]:
let red = new Album('Taylor Swift')

console.log(red.constructor === Album)
console.log(Album.prototype.constructor === Album)
console.log(Album.constructor === Function)

instance
true
true
true


undefined